In [1]:
from google.colab import drive
drive.mount('/content/drive')

directory = '/content/drive/MyDrive/Colab Notebooks/Kollektif_Ogrenme/Proje'

Mounted at /content/drive


In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
# Dataset isim ve directory belirle !!!!!!!!!!!!!!!!!!!!!!

directory = '/content/drive/MyDrive/Colab Notebooks/Kollektif_Ogrenme/Proje'

imdb_train_dataset_filepath = directory+"/imdb_dataset/train_imdb.csv"
imdb_test_dataset_filepath = directory+"/imdb_dataset/test_imdb.csv"

news_train_dataset_filepath = directory+"/news_dataset/train_news.csv"
news_test_dataset_filepath = directory+"/news_dataset/test_news.csv"



imdb_train_dataset_2x_filepath = directory+"/imdb_dataset/train_imdb_2x.csv"
imdb_train_dataset_3x_filepath = directory+"/imdb_dataset/train_imdb_3x.csv"
imdb_train_dataset_5x_filepath = directory+"/imdb_dataset/train_imdb_5x.csv"

imdb_test_dataset_4x_filepath = directory+"/imdb_dataset/test_imdb_4x.csv"
imdb_test_dataset_6x_filepath = directory+"/imdb_dataset/test_imdb_6x.csv"

news_train_dataset_2x_filepath = directory+"/news_dataset/train_news_2x.csv"
news_train_dataset_3x_filepath = directory+"/news_dataset/train_news_3x.csv"
news_train_dataset_5x_filepath = directory+"/news_dataset/train_news_5x.csv"

news_test_dataset_4x_filepath = directory+"/news_dataset/test_news_4x.csv"
news_test_dataset_6x_filepath = directory+"/news_dataset/test_news_6x.csv"

In [17]:
from datasets import Dataset, DatasetDict
import pandas as pd

directory = '/content/drive/MyDrive/Colab Notebooks/Kollektif_Ogrenme/Proje'

# CSV dosyalarından veriyi pandas DataFrame'lerine aktar
train_balanced = pd.read_csv(imdb_train_dataset_filepath)
test_balanced = pd.read_csv(imdb_test_dataset_filepath)

# 5) CSV dosyalarından tekrar okuyarak Hugging Face Dataset'e dönüştürme
train_csv = pd.read_csv(news_train_dataset_filepath)
test_csv = pd.read_csv(news_test_dataset_filepath)



# Pandas DataFrame'lerini Hugging Face Dataset nesnelerine dönüştür
train_ds = Dataset.from_pandas(train_balanced)
test_ds = Dataset.from_pandas(test_balanced)

# Tek bir DatasetDict nesnesinde topla
ds = DatasetDict({
    "train": train_ds,
    "test": test_ds
})

# Veri setini kontrol edelim
print(ds)

train_dataset = Dataset.from_pandas(train_csv)
test_dataset = Dataset.from_pandas(test_csv)

ds2 = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

print(ds2)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 500
    })
})
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 500
    })
})


# tuner007/pegasus_paraphrase

In [ ]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import numpy as np
from collections import Counter
from tqdm import tqdm  # veri işleme sırasında ilerleme durumunu göstermek için kullanılır
import pandas as pd
# Renaming the parameter ds to dataset is consistent with typical usage of "import dataset" from snippet [2].

model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def get_response(input_text, num_return_sequences, num_beams):
    """
    input_text: tek bir metni ifade eder.
    num_return_sequences: Kaç adet paraphrase döndürülecek.
    num_beams: Arama (beam search) genişliği.
    """
    batch = tokenizer(
        [input_text],
        truncation=True,
        padding='longest',
        max_length=60,
        return_tensors="pt"
    ).to(torch_device)

    translated = model.generate(
        **batch,
        max_length=60,
        num_beams=num_beams,
        num_return_sequences=num_return_sequences,
        temperature=1.5
    )
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return tgt_text

def augment_data(ds, split, n):
    """
    ds: Hugging Face DatasetDict nesnesi (örneğin ds["train"] ve ds["test"] bölümleri vardır).
        ds[split] -> Dataset({
            features: ['text', 'label'],
            num_rows: ...
        })

    split: Hangi bölüme (train/test) veri artırımı uygulanacaksa onu belirtir (default 'test').
    n: Her bir satır için üretilecek ek cümle sayısı (3).

    Bu fonksiyon, Dataset'i pandas DataFrame'e çevirir, augment edip geri Dataset'e döndürür.
    Böylece satır sayısı 4 katına çıkar (orijinal + 3 yeni), sütun sayısı aynı (text, label) kalır [[4],[5]].
    """

    # 1) Hugging Face Dataset -> pandas DataFrame (immutable Dataset'i dönüştürme) [[3]]
    df = ds[split].to_pandas()

    new_texts = []
    new_labels = []

    # tqdm ile ilerleme çubuğu
    for idx, row in tqdm(df.iterrows(), total=len(df), desc=f"Augmenting {split} data"):
        orig_text = row["text"]
        orig_label = row["label"]

        # n tane yeni cümle üret [[1],[4]]
        paraphrased_texts = get_response(orig_text, num_return_sequences=n, num_beams=10)

        # Bu yeni cümleleri listeliyoruz
        new_texts.extend(paraphrased_texts)
        new_labels.extend([orig_label] * len(paraphrased_texts))

    # 2) Yeni örnekleri DataFrame'e ekleyelim
    # Orijinal veriler + yeni üretilmiş veriler [[4]]
    new_df = pd.DataFrame({"text": new_texts, "label": new_labels})
    combined_df = pd.concat([df, new_df], ignore_index=True)

    # 3) Tekrar Hugging Face Dataset'e çeviriyoruz
    ds_augmented = Dataset.from_pandas(combined_df)

    # 4) ds içinde ilgili bölümü güncelle
    ds[split] = ds_augmented

    # Kontroller: toplam satır ve label dağılımları
    print(ds)
    print(f"Toplam {split} satır sayısı:", len(ds[split]))
    print(f"Label frekansları ({split}):", Counter(ds[split]["label"]))

    return ds


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at tuner007/pegasus_paraphrase and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Test Augmentation

In [ ]:
print(ds['test']['text'][1])
print("\n")
#ds_4x = augment_data(ds, split='test', n=3)
#ds_4x['test'].to_csv(imdb_test_dataset_4x_filepath, index=False)
#ds_6x = augment_data(ds, split='test', n=5)
#ds_6x['test'].to_csv(imdb_test_dataset_6x_filepath, index=False)

print(ds2['test']['text'][1])
#ds2_4x = augment_data(ds2, split='test', n=3)
#ds2_4x['test'].to_csv(news_test_dataset_4x_filepath, index=False)
ds2_6x = augment_data(ds2, split='test', n=5)
print(ds2_6x['test']['text'][1])
ds2_6x['test'].to_csv(news_test_dataset_6x_filepath, index=False)

If there is such a thing as beautiful horror, this film is one of the best in this genre. It is a horror movie, which despite not being void of gore scenes relies more on psychology and masterful building of the tension in order to create thrills. And it is one of those movies so beautifully filmed, where each scene is a full world of symbols and details, all serving the scope and genre that it can be called but beautiful.<br /><br />It is not an easy story, with two sisters returning to their father and step-mother mansion after having spent some time in a psychiatric institution. They cope hardly with the death of their mother and they try to protect a world of theirs, defending them against the adult world. So the film seems to be at its most external layer. Actually the film slowly evolves to something very different, at slow pace, but no frame is lost to convey the sense of thrilling beauty, so I will not say much more. Watch it, it is one of the best in the genre of Far East horr

Augmenting test data:   0%|          | 0/500 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Augmenting test data: 100%|██████████| 500/500 [06:13<00:00,  1.34it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 3000
    })
})
Toplam test satır sayısı: 3000
Label frekansları (test): Counter({3: 750, 2: 750, 0: 750, 1: 750})
Krispy Kreme Posts Loss, Stock Off 16 Pct Krispy Kreme Doughnuts Inc. (KKD.N: Quote, Profile, Research) on Monday reported a quarterly loss due to store closings and sluggish sales, sending its stock down 16 percent.


Creating CSV from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

441346

## Train Augmentation

In [ ]:
print(ds['train']['text'][1])
print("\n")
"""
ds_2x = augment_data(ds, split='train', n=1)
ds_2x['train'].to_csv(imdb_train_dataset_2x_filepath, index=False)

ds_3x = augment_data(ds, split='train', n=2)
ds_3x['train'].to_csv(imdb_train_dataset_3x_filepath, index=False)

ds_5x = augment_data(ds, split='train', n=4)
ds_5x['train'].to_csv(imdb_train_dataset_5x_filepath, index=False)

print(ds2['train']['text'][1])
ds2_2x = augment_data(ds2, split='train', n=1)
ds2_2x['train'].to_csv(news_train_dataset_2x_filepath, index=False)

ds2_3x = augment_data(ds2, split='train', n=2)
ds2_3x['train'].to_csv(news_train_dataset_3x_filepath, index=False)
"""
ds2_5x = augment_data(ds2, split='train', n=4)
ds2_5x['train'].to_csv(news_train_dataset_5x_filepath, index=False)


Apparently, the people that wrote the back of the box did not bother to watch this so-called "movie." They described "blindingly choreographed intrigue and violence." I saw no "intrigue." I instead saw a miserable attempt at dialogue in a supposed kung fu movie. I saw no "violence." At least, I saw nothing which could cause me to suspend my disbelief as to what could possibly hurt a man with "impervious" skin--but here I am perhaps revealing too much of the "plot." Furthermore, as a viewer of many and sundry films (some of which include the occasional kung fu movie), I can authoritatively say that this piece of celluloid is unwatchable. Whatever you may choose to do, I will always remain<br /><br /> Correct,<br /><br /> Jonathan Tanner <br /><br /> <br /><br />P.S. I was not blinded by the choreography.




Augmenting train data:   0%|          | 0/2000 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Augmenting train data: 100%|██████████| 2000/2000 [21:27<00:00,  1.55it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 500
    })
})
Toplam train satır sayısı: 10000
Label frekansları (train): Counter({1: 2500, 2: 2500, 0: 2500, 3: 2500})


Creating CSV from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

1539344

In [ ]:
a = 1250
b = 4

print(ds2_5x['train']['text'][0+a])
print("\n")
print(ds2_5x['train']['text'][2000+a*b])
print("\n")
print(ds2_5x['train']['text'][2001+a*b])
print("\n")
print(ds2_5x['train']['text'][2002+a*b])
print("\n")
print(ds2_5x['train']['text'][2003+a*b])
print("\n")

Seahawks' Wistrom Suffers Broken Knee  KIRKLAND, Wash. (Sports Network) - Seattle Seahawks  defensive end Grant Wistrom will miss the next four-to-six  weeks due a fracture in his left knee.


(Sports Network) - Seattle Seahawks defensive end Grant Wistrom will miss the next four-to-six weeks due to a fractured knee.


(Sports Network) - Seattle Seahawks defensive end Grant Wistrom will miss the next four-to-six weeks due to a fracture in his left knee.


(Sports Network) - Seattle Seahawks defensive end Grant Wistrom will miss the next four-to-six weeks due to a broken knee.


(Sports Network) - Seattle Seahawks defensive end Grant Wistrom will miss the next four-to-six weeks due to a fractured left knee.




# humarin/chatgpt_paraphraser_on_T5_base

In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

device = "cuda"

tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")

model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

def paraphrase(
    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=5,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=128
):
    input_ids = tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids.to(device)

    outputs = model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )

    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return res


In [9]:
import torch
import numpy as np
from collections import Counter
from tqdm import tqdm
import pandas as pd

split = 'test'



df = ds2[split].to_pandas()

# Paraphrased verilerin ekleneceği listeler
second_texts, second_labels = [], []
third_texts, third_labels = [], []
fourth_texts, fourth_labels = [], []
fifth_texts, fifth_labels = [], []
sixth_texts, sixth_labels = [], []

# tqdm ile ilerleme çubuğu
for idx, row in tqdm(df.iterrows(), total=len(df), desc=f"Augmenting {split} data"):
    orig_text = row["text"]
    orig_label = row["label"]

    # n tane yeni cümle üret [[1],[4]]
    paraphrased_texts = paraphrase(orig_text)

    # Sırasıyla, 2nd, 3rd, 4th, 5th, 6th metin ve label listelerine eklenir
    second_texts.append(paraphrased_texts[0])
    second_labels.append(orig_label)

    third_texts.append(paraphrased_texts[1])
    third_labels.append(orig_label)

    fourth_texts.append(paraphrased_texts[2])
    fourth_labels.append(orig_label)

    fifth_texts.append(paraphrased_texts[3])
    fifth_labels.append(orig_label)

    sixth_texts.append(paraphrased_texts[4])
    sixth_labels.append(orig_label)

# Her bir liste için uzunluk bilgilerini ekrana yazdır
print(f"2nd texts length: {len(second_texts)}")
print(f"3rd texts length: {len(third_texts)}")
print(f"4th texts length: {len(fourth_texts)}")
print(f"5th texts length: {len(fifth_texts)}")
print(f"6th texts length: {len(sixth_texts)}")

# Her bir listenin etiket dağılımlarını ekrana yazdır
print("2nd labels freq:", Counter(second_labels))
print("3rd labels freq:", Counter(third_labels))
print("4th labels freq:", Counter(fourth_labels))
print("5th labels freq:", Counter(fifth_labels))
print("6th labels freq:", Counter(sixth_labels))

Augmenting test data:   0%|          | 0/500 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Augmenting test data: 100%|██████████| 500/500 [11:24<00:00,  1.37s/it]

2nd texts length: 500
3rd texts length: 500
4th texts length: 500
5th texts length: 500
6th texts length: 500
2nd labels freq: Counter({3: 125, 2: 125, 0: 125, 1: 125})
3rd labels freq: Counter({3: 125, 2: 125, 0: 125, 1: 125})
4th labels freq: Counter({3: 125, 2: 125, 0: 125, 1: 125})
5th labels freq: Counter({3: 125, 2: 125, 0: 125, 1: 125})
6th labels freq: Counter({3: 125, 2: 125, 0: 125, 1: 125})


In [10]:
dataset = ds2

second_df = pd.DataFrame({"text": second_texts, "label": second_labels})
combined_df = pd.concat([df, second_df], ignore_index=True)
print(combined_df)


third_df = pd.DataFrame({"text": third_texts, "label": third_labels})
combined_df = pd.concat([combined_df, third_df], ignore_index=True)
print(combined_df)


fourth_df = pd.DataFrame({"text": fourth_texts, "label": fourth_labels})
combined_df = pd.concat([combined_df, fourth_df], ignore_index=True)
print(combined_df)

ds_augmented = Dataset.from_pandas(combined_df)
dataset[split] = ds_augmented
print(dataset)

dataset[split].to_csv(news_test_dataset_4x_filepath, index=False)

fifth_df = pd.DataFrame({"text": fifth_texts, "label": fifth_labels})
combined_df = pd.concat([combined_df, fifth_df], ignore_index=True)
print(combined_df)

sixth_df = pd.DataFrame({"text": sixth_texts, "label": sixth_labels})
combined_df = pd.concat([combined_df, sixth_df], ignore_index=True)
print(combined_df)

ds_augmented = Dataset.from_pandas(combined_df)
dataset[split] = ds_augmented
print(dataset)

dataset[split].to_csv(news_test_dataset_6x_filepath, index=False)

                                                  text  label
0    Microsoft Makes a Play for Novell Users (Ziff ...      3
1    Krispy Kreme Posts Loss, Stock Off 16 Pct Kris...      2
2    Sun launches Solaris 10 Sun Microsystems launc...      3
3    Chief of Halliburton unit wanted top spot John...      2
4    Witnesses: Gunfire at Arafat Mourning Site (AP...      0
..                                                 ...    ...
995  According to Michael Evans of the Food and Dru...      0
996  Despite the existence of an unpatched vulnerab...      3
997  Fatalistic behavior is a result of old-fashion...      1
998  A government official, speaking anonymously, r...      0
999  This week, Borland Software Corp. introduced C...      3

[1000 rows x 2 columns]
                                                   text  label
0     Microsoft Makes a Play for Novell Users (Ziff ...      3
1     Krispy Kreme Posts Loss, Stock Off 16 Pct Kris...      2
2     Sun launches Solaris 10 Sun Microsys

Creating CSV from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

                                                   text  label
0     Microsoft Makes a Play for Novell Users (Ziff ...      3
1     Krispy Kreme Posts Loss, Stock Off 16 Pct Kris...      2
2     Sun launches Solaris 10 Sun Microsystems launc...      3
3     Chief of Halliburton unit wanted top spot John...      2
4     Witnesses: Gunfire at Arafat Mourning Site (AP...      0
...                                                 ...    ...
2495  The FDA is reviewing its guidelines for child ...      0
2496  A significant security flaw in Internet Explor...      3
2497  It's a common trait in old-fashioned ways. I c...      1
2498  The Indian government has launched a successfu...      0
2499  The announcement of CaliberRM 2005, a product ...      3

[2500 rows x 2 columns]
                                                   text  label
0     Microsoft Makes a Play for Novell Users (Ziff ...      3
1     Krispy Kreme Posts Loss, Stock Off 16 Pct Kris...      2
2     Sun launches Solaris 10 

Creating CSV from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

572693

In [ ]:
ds['test'].to_csv(imdb_test_dataset_6x_filepath, index=False)

Creating CSV from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

1343166

In [6]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import numpy as np
from collections import Counter
from tqdm import tqdm
import pandas as pd
from datasets import Dataset

# Örnek amaçlı: ds[split] adlı Dataset içerisindeki veriyi pandas DataFrame'e çeviriyoruz.
split = 'train'
df = ds2[split].to_pandas()

# Paraphrased verilerin ekleneceği listeler
second_texts, second_labels = [], []
third_texts, third_labels = [], []
fourth_texts, fourth_labels = [], []
fifth_texts, fifth_labels = [], []
sixth_texts, sixth_labels = [], []

# Orijinal veriler üzerinde gezinerek paraphrased cümleler üretilir
for idx, row in tqdm(df.iterrows(), total=len(df), desc=f"Augmenting {split} data"):
    orig_text = row["text"]
    orig_label = row["label"]

    # Paraphrase fonksiyonu n tane cümle döndürür
    paraphrased_texts = paraphrase(orig_text)

    # Sırasıyla, 2nd, 3rd, 4th, 5th, 6th metin ve label listelerine eklenir
    second_texts.append(paraphrased_texts[0])
    second_labels.append(orig_label)

    third_texts.append(paraphrased_texts[1])
    third_labels.append(orig_label)

    fourth_texts.append(paraphrased_texts[2])
    fourth_labels.append(orig_label)

    fifth_texts.append(paraphrased_texts[3])
    fifth_labels.append(orig_label)

    #sixth_texts.append(paraphrased_texts[4])
    #sixth_labels.append(orig_label)

# Her bir liste için uzunluk bilgilerini ekrana yazdır
print(f"2nd texts length: {len(second_texts)}")
print(f"3rd texts length: {len(third_texts)}")
print(f"4th texts length: {len(fourth_texts)}")
print(f"5th texts length: {len(fifth_texts)}")
#print(f"6th texts length: {len(sixth_texts)}")

# Her bir listenin etiket dağılımlarını ekrana yazdır
print("2nd labels freq:", Counter(second_labels))
print("3rd labels freq:", Counter(third_labels))
print("4th labels freq:", Counter(fourth_labels))
print("5th labels freq:", Counter(fifth_labels))
#print("6th labels freq:", Counter(sixth_labels))

# İsteğe bağlı olarak, yeni verileri DataFrame'e ekleyip concatenation yapabilirsiniz
# .concat() fonksiyonu, Pandas’ta farklı DataFrame'leri birleştirmek için kullanılır [[3]]



Augmenting train data:   0%|          | 0/2000 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Augmenting train data: 100%|██████████| 2000/2000 [45:16<00:00,  1.36s/it]

2nd texts length: 2000
3rd texts length: 2000
4th texts length: 2000
5th texts length: 2000
2nd labels freq: Counter({1: 500, 2: 500, 0: 500, 3: 500})
3rd labels freq: Counter({1: 500, 2: 500, 0: 500, 3: 500})
4th labels freq: Counter({1: 500, 2: 500, 0: 500, 3: 500})
5th labels freq: Counter({1: 500, 2: 500, 0: 500, 3: 500})


In [7]:
dataset = ds2

second_df = pd.DataFrame({"text": second_texts, "label": second_labels})
combined_df = pd.concat([df, second_df], ignore_index=True)
print(combined_df)

ds_augmented = Dataset.from_pandas(combined_df)
dataset[split] = ds_augmented
print(dataset)

dataset[split].to_csv(news_train_dataset_2x_filepath, index=False)




third_df = pd.DataFrame({"text": third_texts, "label": third_labels})
combined_df = pd.concat([combined_df, third_df], ignore_index=True)
print(combined_df)

ds_augmented = Dataset.from_pandas(combined_df)
dataset[split] = ds_augmented
print(dataset)

dataset[split].to_csv(news_train_dataset_3x_filepath, index=False)




fourth_df = pd.DataFrame({"text": fourth_texts, "label": fourth_labels})
combined_df = pd.concat([combined_df, fourth_df], ignore_index=True)
print(combined_df)

fifth_df = pd.DataFrame({"text": fifth_texts, "label": fifth_labels})
combined_df = pd.concat([combined_df, fifth_df], ignore_index=True)
print(combined_df)

ds_augmented = Dataset.from_pandas(combined_df)
dataset[split] = ds_augmented
print(dataset)

dataset[split].to_csv(news_train_dataset_5x_filepath, index=False)

                                                   text  label
0     Allenby predicts Masters surge Defending champ...      1
1     Crematory to Draw Guilty Plea Nearly three yea...      2
2     Fannie Mae Investigation Lawrence J. White, pr...      2
3     Astros Grab Wild Card The Astros charge into t...      1
4     Orbitz Profit Soars on Revenue Surge Orbitz In...      2
...                                                 ...    ...
3995  Wal-Mart Stores Inc., the largest retailer glo...      2
3996  Despite Arsenal's satisfaction with their 100 ...      1
3997  Sources close to the ECB have revealed that th...      2
3998  Sinn Fein president Gerry Adams spoke on the 3...      0
3999  Guillermo Canas secures his place in the Shang...      1

[4000 rows x 2 columns]
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 500
    })
})


Creating CSV from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

                                                   text  label
0     Allenby predicts Masters surge Defending champ...      1
1     Crematory to Draw Guilty Plea Nearly three yea...      2
2     Fannie Mae Investigation Lawrence J. White, pr...      2
3     Astros Grab Wild Card The Astros charge into t...      1
4     Orbitz Profit Soars on Revenue Surge Orbitz In...      2
...                                                 ...    ...
5995  On Monday, Wal-Mart Stores Inc., the largest r...      2
5996  Arsenal's 100 million deal with Emirates was o...      1
5997  The European Central Bank's growth forecast fo...      2
5998  The Irish Republican Army #39's political fact...      0
5999  Jiri Novak is out of the competition as Guille...      1

[6000 rows x 2 columns]
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 6000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 500
    })
})


Creating CSV from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

                                                   text  label
0     Allenby predicts Masters surge Defending champ...      1
1     Crematory to Draw Guilty Plea Nearly three yea...      2
2     Fannie Mae Investigation Lawrence J. White, pr...      2
3     Astros Grab Wild Card The Astros charge into t...      1
4     Orbitz Profit Soars on Revenue Surge Orbitz In...      2
...                                                 ...    ...
7995  The world's largest retailer, Wal-Mart Stores ...      2
7996  Emirates reported that Arsenal's 100 million d...      1
7997  The ECB has updated its growth forecast for th...      2
7998  In 1974, a number of pub bombings took place i...      0
7999  The Shanghai Open finals saw Guillermo Canas c...      1

[8000 rows x 2 columns]
                                                   text  label
0     Allenby predicts Masters surge Defending champ...      1
1     Crematory to Draw Guilty Plea Nearly three yea...      2
2     Fannie Mae Investigation

Creating CSV from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

1945855

In [ ]:
text = 'What are the best places to see in New York?'
a = paraphrase(text)
print(a)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


['What are some must-see places in New York?', 'Can you suggest some must-see spots in New York?', 'Where should one go to experience the best NYC has to offer?']


# mrm8488/t5-small-finetuned-quora-for-paraphrasing

In [14]:
from transformers import AutoModelWithLMHead, AutoTokenizer

device = "cuda"

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-small-finetuned-quora-for-paraphrasing")
model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-small-finetuned-quora-for-paraphrasing").to(device)

def paraphrase(text, max_length=128):

  input_ids = tokenizer.encode(text, return_tensors="pt", add_special_tokens=True)

  input_ids = input_ids.to(device)

  generated_ids = model.generate(input_ids=input_ids, num_return_sequences=4, num_beams=5, max_length=max_length, no_repeat_ngram_size=2, repetition_penalty=3.5, length_penalty=1.0, early_stopping=True)

  preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]

  return preds

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1838: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


In [18]:
import torch
import numpy as np
from collections import Counter
from tqdm import tqdm
import pandas as pd

split = 'train'



df = ds2[split].to_pandas()

# Paraphrased verilerin ekleneceği listeler
second_texts, second_labels = [], []
third_texts, third_labels = [], []
fourth_texts, fourth_labels = [], []
fifth_texts, fifth_labels = [], []
#sixth_texts, sixth_labels = [], []

# tqdm ile ilerleme çubuğu
for idx, row in tqdm(df.iterrows(), total=len(df), desc=f"Augmenting {split} data"):
    orig_text = row["text"]
    orig_label = row["label"]

    # n tane yeni cümle üret [[1],[4]]
    paraphrased_texts = paraphrase(orig_text)

    # Sırasıyla, 2nd, 3rd, 4th, 5th, 6th metin ve label listelerine eklenir
    second_texts.append(paraphrased_texts[0])
    second_labels.append(orig_label)

    third_texts.append(paraphrased_texts[1])
    third_labels.append(orig_label)

    fourth_texts.append(paraphrased_texts[2])
    fourth_labels.append(orig_label)

    fifth_texts.append(paraphrased_texts[3])
    fifth_labels.append(orig_label)

    #sixth_texts.append(paraphrased_texts[4])
    #sixth_labels.append(orig_label)

# Her bir liste için uzunluk bilgilerini ekrana yazdır
print(f"2nd texts length: {len(second_texts)}")
print(f"3rd texts length: {len(third_texts)}")
print(f"4th texts length: {len(fourth_texts)}")
print(f"5th texts length: {len(fifth_texts)}")
#print(f"6th texts length: {len(sixth_texts)}")

# Her bir listenin etiket dağılımlarını ekrana yazdır
print("2nd labels freq:", Counter(second_labels))
print("3rd labels freq:", Counter(third_labels))
print("4th labels freq:", Counter(fourth_labels))
print("5th labels freq:", Counter(fifth_labels))
#print("6th labels freq:", Counter(sixth_labels))

Augmenting train data: 100%|██████████| 2000/2000 [21:54<00:00,  1.52it/s]

2nd texts length: 2000
3rd texts length: 2000
4th texts length: 2000
5th texts length: 2000
2nd labels freq: Counter({1: 500, 2: 500, 0: 500, 3: 500})
3rd labels freq: Counter({1: 500, 2: 500, 0: 500, 3: 500})
4th labels freq: Counter({1: 500, 2: 500, 0: 500, 3: 500})
5th labels freq: Counter({1: 500, 2: 500, 0: 500, 3: 500})


## Test Augmantation

In [10]:
dataset = ds2

second_df = pd.DataFrame({"text": second_texts, "label": second_labels})
combined_df = pd.concat([df, second_df], ignore_index=True)
print(combined_df)


third_df = pd.DataFrame({"text": third_texts, "label": third_labels})
combined_df = pd.concat([combined_df, third_df], ignore_index=True)
print(combined_df)


fourth_df = pd.DataFrame({"text": fourth_texts, "label": fourth_labels})
combined_df = pd.concat([combined_df, fourth_df], ignore_index=True)
print(combined_df)

ds_augmented = Dataset.from_pandas(combined_df)
dataset[split] = ds_augmented
print(dataset)

dataset[split].to_csv(news_test_dataset_4x_filepath, index=False)

fifth_df = pd.DataFrame({"text": fifth_texts, "label": fifth_labels})
combined_df = pd.concat([combined_df, fifth_df], ignore_index=True)
print(combined_df)

sixth_df = pd.DataFrame({"text": sixth_texts, "label": sixth_labels})
combined_df = pd.concat([combined_df, sixth_df], ignore_index=True)
print(combined_df)

ds_augmented = Dataset.from_pandas(combined_df)
dataset[split] = ds_augmented
print(dataset)

dataset[split].to_csv(news_test_dataset_6x_filepath, index=False)

                                                  text  label
0    Microsoft Makes a Play for Novell Users (Ziff ...      3
1    Krispy Kreme Posts Loss, Stock Off 16 Pct Kris...      2
2    Sun launches Solaris 10 Sun Microsystems launc...      3
3    Chief of Halliburton unit wanted top spot John...      2
4    Witnesses: Gunfire at Arafat Mourning Site (AP...      0
..                                                 ...    ...
995  What are the warning labels for anti-depressan...      0
996  Microsoft issues one fix in monthly security u...      3
997  What is the old frame of mind quot;Hey, there....      1
998  On Friday, a government official said on condi...      0
999  CaliberRM Borland adds Project Management Feat...      3

[1000 rows x 2 columns]
                                                   text  label
0     Microsoft Makes a Play for Novell Users (Ziff ...      3
1     Krispy Kreme Posts Loss, Stock Off 16 Pct Kris...      2
2     Sun launches Solaris 10 Sun Microsys

Creating CSV from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

                                                   text  label
0     Microsoft Makes a Play for Novell Users (Ziff ...      3
1     Krispy Kreme Posts Loss, Stock Off 16 Pct Kris...      2
2     Sun launches Solaris 10 Sun Microsystems launc...      3
3     Chief of Halliburton unit wanted top spot John...      2
4     Witnesses: Gunfire at Arafat Mourning Site (AP...      0
...                                                 ...    ...
2495  What are the warning labels for anti-depressan...      0
2496  Microsoft issues one fix in monthly security u...      3
2497       You look happy. Red Sox fever still got you?      1
2498  On Friday, a government official said on condi...      0
2499  CaliberRM Borland adds Project Management Feat...      3

[2500 rows x 2 columns]
                                                   text  label
0     Microsoft Makes a Play for Novell Users (Ziff ...      3
1     Krispy Kreme Posts Loss, Stock Off 16 Pct Kris...      2
2     Sun launches Solaris 10 

Creating CSV from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

600764

## Train Augmentation

In [19]:
dataset = ds2

second_df = pd.DataFrame({"text": second_texts, "label": second_labels})
combined_df = pd.concat([df, second_df], ignore_index=True)
print(combined_df)

ds_augmented = Dataset.from_pandas(combined_df)
dataset[split] = ds_augmented
print(dataset)

dataset[split].to_csv(news_train_dataset_2x_filepath, index=False)




third_df = pd.DataFrame({"text": third_texts, "label": third_labels})
combined_df = pd.concat([combined_df, third_df], ignore_index=True)
print(combined_df)

ds_augmented = Dataset.from_pandas(combined_df)
dataset[split] = ds_augmented
print(dataset)

dataset[split].to_csv(news_train_dataset_3x_filepath, index=False)




fourth_df = pd.DataFrame({"text": fourth_texts, "label": fourth_labels})
combined_df = pd.concat([combined_df, fourth_df], ignore_index=True)
print(combined_df)

fifth_df = pd.DataFrame({"text": fifth_texts, "label": fifth_labels})
combined_df = pd.concat([combined_df, fifth_df], ignore_index=True)
print(combined_df)

ds_augmented = Dataset.from_pandas(combined_df)
dataset[split] = ds_augmented
print(dataset)

dataset[split].to_csv(news_train_dataset_5x_filepath, index=False)

                                                   text  label
0     Allenby predicts Masters surge Defending champ...      1
1     Crematory to Draw Guilty Plea Nearly three yea...      2
2     Fannie Mae Investigation Lawrence J. White, pr...      2
3     Astros Grab Wild Card The Astros charge into t...      1
4     Orbitz Profit Soars on Revenue Surge Orbitz In...      2
...                                                 ...    ...
3995  Wal-Mart Brightens Dec. Sales Forecast (Reuter...      2
3996  Arsenal lost 23M in Emirates deal The Gunners ...      1
3997  BRUSSELS - The European Central Bank has lower...      2
3998  On the 30th anniversary of the killings, Sinn ...      0
3999  Guillermo Canas beat Jiri Novak 6-1, 6-2 in th...      1

[4000 rows x 2 columns]
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 500
    })
})


Creating CSV from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

                                                   text  label
0     Allenby predicts Masters surge Defending champ...      1
1     Crematory to Draw Guilty Plea Nearly three yea...      2
2     Fannie Mae Investigation Lawrence J. White, pr...      2
3     Astros Grab Wild Card The Astros charge into t...      1
4     Orbitz Profit Soars on Revenue Surge Orbitz In...      2
...                                                 ...    ...
5995  Wal-Mart Brightens Dec. Sales Forecast (Reuter...      2
5996  Arsenal lost 23M in Emirates deal The Gunners ...      1
5997  BRUSSELS - The European Central Bank has lower...      2
5998  On the 30th anniversary of the killings, Sinn ...      0
5999  Guillermo Canas beat Jiri Novak 6-1, 6-2 in th...      1

[6000 rows x 2 columns]
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 6000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 500
    })
})


Creating CSV from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

                                                   text  label
0     Allenby predicts Masters surge Defending champ...      1
1     Crematory to Draw Guilty Plea Nearly three yea...      2
2     Fannie Mae Investigation Lawrence J. White, pr...      2
3     Astros Grab Wild Card The Astros charge into t...      1
4     Orbitz Profit Soars on Revenue Surge Orbitz In...      2
...                                                 ...    ...
7995  Wal-Mart Brightens Dec. Sales Forecast (Reuter...      2
7996  Arsenal lost 23m in Emirates deal The Gunners ...      1
7997  BRUSSELS - The European Central Bank has lower...      2
7998  The political arm of the Irish Republican Army...      0
7999  Guillermo Canas beat Jiri Novak 6-1, 6-2 in th...      1

[8000 rows x 2 columns]
                                                   text  label
0     Allenby predicts Masters surge Defending champ...      1
1     Crematory to Draw Guilty Plea Nearly three yea...      2
2     Fannie Mae Investigation

Creating CSV from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

2016438